# New case studies for Robot Dance paper

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime

import run_robot
import prepare_data
from importlib import reload
reload(prepare_data)
reload(run_robot)

In [ ]:
# Define the basic data for the case studies

# Compute initial values for first run and read the cities and other initial data
# basic_prm = prepare_data.save_basic_parameters(min_level=0.8, rep=2.0, ndays=(180 + 28), tinf=2.9, need_icu=0.00379872899804252, time_icu=11)
basic_prm = prepare_data.save_basic_parameters(min_level=0.8, rep=2.0, ndays=(180 + 28), tinf=2.9, need_icu=0.00693521103887298, time_icu=7)

# Compute initial values and read mobility matrix

# For cities
# cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", ["Araçatuba", "São José Do Rio Preto"], 500000, 1)
# cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", ["São José Do Rio Preto"], 25000, 6, 1)
# M = prepare_data.convert_mobility_matrix_and_save(cities_data, max_neighbors=3, drs=False)

# For DRS
subnot_factor = 11.6
cities_data = prepare_data.compute_initial_condition_evolve_and_save(basic_prm, "SP", [], 1, subnot_factor, 1, "data/covid_with_drs.csv")
M = prepare_data.convert_mobility_matrix_and_save(cities_data, max_neighbors=0, drs=True)

In [ ]:
cities_data = pd.read_csv("data/cities_data.csv", index_col=0)

# Create a target matrix (max infected level)
ncities, ndays = len(cities_data.index), int(basic_prm["ndays"])
target = np.ones((ncities, ndays))
target = prepare_data.save_target(cities_data, target)
hammer_data = prepare_data.save_hammer_data(cities_data, 0, 0.8)

# Use a forcedif that releases the cities in the end
force_dif = np.ones((ncities, ndays))
cities_data

## Case 1: 14 day window, official data

In [ ]:
verbosity = 2
basic_prm["alternate"] = 0.0
budget = 50000000
run_robot.find_feasible_hammer(basic_prm, cities_data, M, target, hammer_data, None, incr_all=True, verbosity=verbosity)
run_robot.prepare_optimization(basic_prm, cities_data, M, target, hammer_data, force_dif, verbosity=verbosity, test_budget=budget)

In [ ]:
%%time
run_robot.optimize_and_show_results(basic_prm, f"results/tests_new_{budget:d}_daily_unlimited_no_mobility_sp.png", f"results/tests_new_{budget:d}_daily_unlimited_no_mobility_sp.csv", 
                                    cities_data, verbosity=verbosity)